<font size="+3"><strong>Air Quality in Dar es Salaam 🇹🇿</strong></font>

# Import Necessary Libraries

In [1]:
# # Import libraries here

# import inspect
# import time

# import matplotlib.pyplot as plt
# import pandas as pd
# import plotly.express as px
# import seaborn as sns
# from IPython.display import VimeoVideo
# from pymongo import MongoClient
# from sklearn.metrics import mean_absolute_error
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa.arima.model import ARIMA

# Select Specified Database

In [2]:
# client = MongoClient(host="localhost", port=27017)
# db = client["air-quality"]
# # print(db)
# print(list(db.list_collections())[1])
# for c in db.list_collections():
#     print(c["name"])
# dar = db["dar-es-salaam"]

# Get Distinct Values

In [3]:
# # print(dar.find({})[0])
# # sites 
# sites = dar.distinct("metadata.site")

# list(sites)

# Do aggregation

In [4]:
# result = dar.aggregate(
#     [
#         {"$group": 
#              {
#                  "_id": "$metadata.site", 
#                  "count":{"$count":{}}
#              }
#         }
#     ]
# )
# readings_per_site = list(result)
# readings_per_site

# Wrangle Data

In [5]:
# def wrangle(collection, resample_rule = "1H"):

#     results = collection.find(
#         {"metadata.site": 11, "metadata.measurement": "P2"},
#         projection={"P2": 1, "timestamp": 1, "_id": 0},
#     )
# #     results = list(collection)

#     # Read results into DataFrame
#     df = pd.DataFrame(list(results)).set_index("timestamp")

#     # Localize timezone
#     df.index = df.index.tz_localize("UTC").tz_convert("Africa/Dar_es_Salaam")

#     # Remove outliers
#     df = df[df["P2"] <= 100]

#     # Resample and forward-fill
#     y = df["P2"].resample(resample_rule).mean().fillna(method='ffill').to_frame()["P2"]

#     return y

In [6]:
# y = wrangle(dar)
# y.head()

# Plot readings with Line Plot

In [7]:
# fig, ax = plt.subplots(figsize=(15, 6))
# y.plot(ax = ax)
# plt.xlabel("Date")
# plt.ylabel("PM2.5 Level")
# plt.title("Dar es Salaam PM2.5 Levels")

# # Don't delete the code below 👇
# plt.savefig("images/3-5-5.png", dpi=150)


# Create a plot of rolling values

In [8]:
# fig, ax = plt.subplots(figsize=(15, 6))

# plt.plot(y.rolling(168).mean())
# plt.xlabel("Date")
# plt.ylabel("PM2.5 Level")
# plt.title("Dar es Salaam PM2.5 Levels, 7-Day Rolling Average")
# # Don't delete the code below 👇
# plt.savefig("images/3-5-6.png", dpi=150)


# Plot ACF

In [9]:
# fig, ax = plt.subplots(figsize=(15, 6))

# plot_acf(y, ax= ax)
# plt.xlabel("Lag [hours]")
# plt.ylabel("Correlation Coefficient")
# plt.title("Dar es Salaam PM2.5 Readings, ACF")
# # Don't delete the code below 👇
# plt.savefig("images/3-5-7.png", dpi=150)


# Plot PACF

In [10]:
# fig, ax = plt.subplots(figsize=(15, 6))
# plot_pacf(y, ax= ax)
# plt.xlabel("Lag [hours]")
# plt.ylabel("Correlation Coefficient")
# plt.title("Dar es Salaam PM2.5 Readings, PACF")
# # Don't delete the code below 👇
# plt.savefig("images/3-5-8.png", dpi=150)


# Split Dataset

In [11]:
# cutoff_test = int(len(y) * 0.9)
# y_train = y[:cutoff_test]
# y_test = y[cutoff_test:]
# print("y_train shape:", y_train.shape)
# print("y_test shape:", y_test.shape)

# Build Baseline Model

In [12]:
# y_train_mean = y_train.mean()
# y_pred_baseline = [y_train_mean] * len(y_train)
# mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

# print("Mean P2 Reading:", y_train_mean)
# print("Baseline MAE:", mae_baseline)

# Create Autoregressive Model

In [13]:
# from statsmodels.tsa.ar_model import AutoReg

# p_params = range(1, 31)
# maes = []
# x_type = None
# for p in p_params:
#     model = AutoReg(y_train, lags= p).fit()
#     x_type = type(model)
#     y_pred = model.predict()
#     training_mae = mean_absolute_error(y_train[p:], y_pred[p:])
#     maes.append(training_mae)

# print(x_type)
# mae_series = pd.Series(maes, name="mae", index=p_params)
# mae_series.head()

### Select best p parameter for the model

In [14]:
# best_p = mae_series[mae_series == mae_series.max()].index[0]
# print(best_p)
# best_model = AutoReg(y_train, lags= best_p).fit()

### Select best model residuals


In [15]:
# y_train_resid = best_model.resid
# y_train_resid.name = "residuals"
# y_train_resid.head()

### See distribution of residuals

In [16]:
# # Plot histogram of residuals
# y_train_resid.plot(kind = "hist")
# plt.xlabel("Residuals")
# plt.ylabel("Frequency")
# plt.title("Best Model, Training Residuals")

# # Don't delete the code below 👇
# plt.savefig("images/3-5-14.png", dpi=150)


### See ACF of model residuals

In [17]:
# fig, ax = plt.subplots(figsize=(15, 6))
# plot_acf(y_train_resid, ax =ax)

# plt.xlabel("Lag [hours]")
# plt.ylabel("Correlation Coefficient")
# plt.title("Dar es Salaam, Training Residuals ACF")
# # Don't delete the code below 👇
# plt.savefig("images/3-5-15.png", dpi=150)


# Walk-Forward Validation

In [18]:
# y_pred_wfv = pd.Series(dtype='float64')
# history = y_train.copy()
# for i in range(len(y_test)):
#     model = AutoReg(history, lags= 26).fit()
#     next_pred = model.forecast()
#     y_pred_wfv = y_pred_wfv.append(next_pred)
#     history = history.append(y_test[next_pred.index])
    
# y_pred_wfv.name = "prediction"
# y_pred_wfv.index.name = "timestamp"
# y_pred_wfv.head()

# Communicate Results

In [19]:
# df_pred_test = pd.DataFrame({
#     "y_test":y_test,
#     "y_pred_wfv":y_pred_wfv
# })
# fig = px.line(df_pred_test)
# fig.update_layout(
#     title="Dar es Salaam, WFV Predictions",
#     xaxis_title="Date",
#     yaxis_title="PM2.5 Level",
# )
# # Don't delete the code below 👇
# fig.write_image("images/3-5-18.png", scale=1, height=500, width=700)

# fig.show()